In [ ]:
import omnipath as op
import pandas as pd
import networkx as nx
import numpy as np
import scipy
import torch
import torch_geometric
from torch_geometric.data import Data
import sklearn
import sys
from run import graphscattering as gs

## Run Directed Scattering on training graph

In [ ]:
np.random.seed(0)

for dataset in ['Texas', 'Cornell', 'omnipath', 'SIGNOR', 'iPTMnet']:
    print (dataset)
    
    splits = np.load(f'data/{dataset}_curated_interactions.npz')
    train, val, test = splits['train'], splits['val'], splits['test']
    train = pd.DataFrame(train)
    train.columns = ['source', 'target']

    G = nx.from_pandas_edgelist(train, source='source', target='target', create_using=nx.DiGraph)
    A = nx.adjacency_matrix(G).toarray()

    N = A.shape[0]
    signal = np.random.randn(N, 1)
    for q in [0.0, 0.1, 0.2]:
        vals, vecs = gs.compute_eigen(A, q)
        for J in [5, 10, 15]:
            print ('J', J)
            scales = np.flip(np.arange(0, J+1))
            all_features, f = gs.compute_all_features(vals, vecs, signal, N, "lowpass", scales)

            train_ds = pd.DataFrame(data=all_features, index=G.nodes())
            train_ds.to_csv(f'results/Directed_Scattering/Directed_Scattering_J{J}_q{q}_{dataset}_embedding.csv', compression='gzip')

## Run Directed Scattering on train+val graph


In [ ]:
np.random.seed(0)

for dataset in ['Texas', 'Cornell', 'omnipath', 'SIGNOR', 'iPTMnet']:
    print (dataset)
    
    splits = np.load(f'data/{dataset}_curated_interactions.npz')
    train, val, test = splits['train'], splits['val'], splits['test']
    train = pd.DataFrame(np.vstack((train, val)))
    train.columns = ['source', 'target']

    G = nx.from_pandas_edgelist(train, source='source', target='target', create_using=nx.DiGraph)
    A = nx.adjacency_matrix(G).toarray()

    N = A.shape[0]
    signal = np.random.randn(N, 1)
    for q in [0.0, 0.1, 0.2]:
        print ('q', q)
        vals, vecs = gs.compute_eigen(A, q)
        for J in [5, 10, 15]:
            print ('J', J)
            scales = np.flip(np.arange(0, J+1))
            all_features, f = gs.compute_all_features(vals, vecs, signal, N, "lowpass", scales)

            train_ds = pd.DataFrame(data=all_features, index=G.nodes())
            train_ds.to_csv(f'results/Directed_Scattering/Directed_Scattering_J{J}_q{q}_{dataset}_train_val_embedding.csv', compression='gzip')